In [1]:
from PIL import Image
from feature_extractor import FeatureExtractor
from pathlib import Path
import numpy as np
from elasticsearch import Elasticsearch
from os import listdir
from os.path import isfile, join
import argparse
import json
import pathlib

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [3]:
es.cluster.health(wait_for_status='yellow', request_timeout=1)

C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_15644\1636688961.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.cluster.health(wait_for_status='yellow', request_timeout=1)
C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_15644\1636688961.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.cluster.health(wait_for_status='yellow', request_timeout=1)


ObjectApiResponse({'cluster_name': 'elasticsearch', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 15, 'active_shards': 15, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 1, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 93.75})

In [4]:
index = 'content-images'
source_no_vecs = ['raw_data', 'image_path']

In [5]:
def display_hits(res):
    print(f"Found {res['hits']['total']['value']} hits in {res['took']} ms. Showing top {len(res['hits']['hits'])}.")
    print("")
    for hit in res['hits']['hits']:
        s = hit['_source']
        print(f"Title   {s.get('title', None)}")
        if 'tags' in s:
            desc = str(s.get('tags', None))
            print(f"Desc    {desc[:80] + ('...' if len(desc) > 80 else '')}")
        if 'price' in s:
            print(f"Labels   {s['labels']}")
        print(f"ID      {s.get('imageId', None)}")
        print(f"Score   {hit.get('_score', None)}")
        display(Image(s.get("imgUrl"), width=128))
        print("")

In [19]:
def search_by_image_query(image_id=None, feature_vector=None,size=5):
    if image_id==None and feature_vector==None:
        raise ValueError("Please enter an Image ID or a Feature Vector")
    
#     if image_id:
#         query = {
#             "elastiknn_nearest_neighbors": {
#               "vec": {
#                 "index": index,
#                 "field": "featureVec",
#                 "id": image_id
#               },
#               "field": "featureVec",
#               "model": "lsh",
#               "similarity": "l2",
#               "candidates": 100
#             }
#           }
#     elif feature_vector:
    query = {
        "elastiknn_nearest_neighbors": {
          "vec": {
            "values": feature_vector
          },
          "field": "raw_data",
          "model": "lsh",
          "similarity": "l2",
          "candidates": 150
        }
      }
    
    res = es.search(index=index, query=query, size=size, _source=source_no_vecs)
    display_hits(res)

In [7]:
fe = FeatureExtractor()

In [8]:
img = Image.open("C:/Users/KaryGauss/Downloads/640px-Ryanair.arp.750pix.jpg")

In [9]:
features = fe.extract(img)

1/1 [==============================] - 1s 854ms/step


In [56]:
QUERY = """
{
"_source": ["image_path", "raw_data"],
"query": {
  "function_score" : {
    "query" : {
      "match_all" : {
        "boost" : 1.0
      }
    },
    "functions" : [
      {
        "filter" : {
          "match_all" : {
            "boost" : 1.0
          }
        },
        "script_score" : {
          "script" : {
            "inline" : "hamming_score",
            "lang" : "native",
            "params" : {
              "f" : "raw_data",
              "fea" : [##fea##],
              "verbose" : true
            }
          }
        }
      }
    ],
    "score_mode" : "sum",
    "boost_mode" : "replace",
    "max_boost" : 3.4028235E38,
    "boost" : 1.0
  }
}
}
"""

In [77]:
str_feat = str(features)
query = query2.replace('##fea##', str_feat[1:len(str_feat) - 1])

print(query)


{ 
    "query": {
  "function_score" : {
    "query" : {
      "match_all" : {
        "boost" : 1.0
      }
    },
    "functions" : [
      {
        "filter" : {
          "match_all" : {
            "boost" : 1.0
          }
        },
        "script_score" : {
          "script" : {
            "inline" : "hamming_score",
            "lang" : "native",
            "params" : {
              "f" : "raw_data",
              "fea" : [0.06408552 0.         0.         ... 0.         0.         0.        ],
              "verbose" : True
            }
          }
        }
      }
    ],
    "score_mode" : "sum",
    "boost_mode" : "replace",
    "max_boost" : 3.4028235E38,
    "boost" : 1.0
  }
}
}



In [78]:
query2 = """
{ 
    "query": {
  "function_score" : {
    "query" : {
      "match_all" : {
        "boost" : 1.0
      }
    },
    "functions" : [
      {
        "filter" : {
          "match_all" : {
            "boost" : 1.0
          }
        },
        "script_score" : {
          "script" : {
            "inline" : "hamming_score",
            "lang" : "native",
            "params" : {
              "f" : "raw_data",
              "fea" : [##fea##],
              "verbose" : True
            }
          }
        }
      }
    ],
    "score_mode" : "sum",
    "boost_mode" : "replace",
    "max_boost" : 3.4028235E38,
    "boost" : 1.0
  }
}
}
"""

In [79]:
body = {
    "query": {
        "hnsw": {
            "raw_data": {
                "string": str(features),
                "size": 5,
                "ef": 10
            }
        }
    },
    "_source": {"includes": ["image_path"]},
    "from": 0,
    "size": 40
}

In [81]:
result = es.search(index=index, body = body)

C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_15644\3270713796.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index=index, body = body)


BadRequestError: BadRequestError(400, 'parsing_exception', 'unknown query [hnsw]')